In [1]:
from keras import backend as K
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping

import pandas as pd
import numpy as np
import numpy
import tensorflow as tf
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

# seed 값 설정
seed = 0
numpy.random.seed(seed)
tf.set_random_seed(seed)


features =20
filename="./dataset/20200605_v1.csv"

# 데이터 읽어 들이기--- (※1)
#ci_year	ci_term	si_grad	si_ent_gubun	agescale	si_sex	si_last_hak	schoarYn	
#accept_point	avg_logincnt	avg_att_time	avg_att_rate	avg_bbscnt	avg_startlogin	entYn	classcnt	
#avg_te_avg	avg_te_passrate	gradcnt	counsel	stdResult

df_pre=pd.read_csv(filename,encoding="cp949", header=0) # CSV파일을 불러오는 함수를 이용
#df_pre=pd.read_csv('./dataset/20200612_app_point.csv',encoding="cp949", header=0) # CSV파일을 불러오는 함수를 이용

#df_pre_new=df_pre[df_pre.stdResult!=3] #졸업생은 제외



Using TensorFlow backend.
C:\programs\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\programs\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\programs\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\programs\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: Future

In [2]:

df_pre


,ci_year,ci_term,si_grad,si_ent_gubun,agescale,si_sex,si_last_hak,schoarYn,accept_point,avg_logincnt,...,avg_att_rate,avg_bbscnt,avg_startlogin,entYn,classcnt,pre_avg_te_avg,pre_avg_te_passrate,pre_gradcnt,counsel,stdResult
0,2015,1,4,1,60,1,1,1,0,87,...,99.56,0,38,0,5,3.50,100,7,0,1
1,2015,1,4,1,30,2,1,1,0,26,...,85.52,0,18,0,4,2.00,100,10,0,1
2,2015,1,4,1,40,2,2,0,0,26,...,100.00,0,13,0,3,2.50,100,10,0,1
3,2015,1,4,1,20,1,2,0,0,8,...,84.21,0,5,0,6,2.00,100,8,0,1
4,2015,1,4,1,30,1,2,0,0,5,...,81.58,0,4,0,1,0.00,0,12,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58083,2019,2,4,2,50,1,3,1,70,74,...,100.00,0,30,0,6,3.42,100,1,0,0
58084,2019,2,4,2,50,2,3,1,70,50,...,99.72,0,20,0,8,3.67,100,1,0,0
58085,2019,2,4,2,40,2,3,1,70,63,...,100.00,6,22,0,6,3.33,100,1,0,0
58086,2019,2,4,2,40,2,2,1,70,98,...,100.00,12,47,0,8,3.67,100,1,0,0


In [3]:
# 데이터 내부의 기호를 숫자로 변환하기--- (※2)
df = df_pre.sample(frac=1)

dataset = df.values
X = dataset[:,0:features]
Y = dataset[:,features]

# 학습 전용과 테스트 전용 데이터로 나누기 --- (※3)
#테스트, 검증데이터 분할 7:3 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=seed)

#데이터 정규화
X_train, X_test = X_train / 255, X_test / 255

from keras.layers import Dense, Dropout, Activation
# 모델 설정
model = Sequential()
model.add(Dense(28, input_dim=features, activation='relu'))
model.add(Dense(15, activation='relu'))
model.add(Dense(5, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

#모델 컴파일
model.compile(loss='binary_crossentropy',
           optimizer='adam',
           # metrics=['accuracy'])
           metrics=['accuracy',recall_m,precision_m,f1_m])

# 학습 자동 중단 설정
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=100)

# 모델 실행 및 저장
history = model.fit(X_train, Y_train, validation_split=0.20, epochs=500, batch_size=100, 
                    callbacks=[early_stopping_callback])


# 결과 출력
print("\n epoches-500, bat_size=100")
print("\n 학습중단 + 모델 성능개선 : arly_stopping_callback:")
print("\n 예측정확도: %.4f" % (model.evaluate(X_test, Y_test)[1]))


#테스트 데이터 검증
loss, accuracy, recall,precision,f1_socre  = model.evaluate(X_test, Y_test)
#accuracy  = model.evaluate(X_test, Y_test)

print("\n accuracy, precision, recall, f1_score")
print("\n  %.4f, %.4f, %4f, %.4f" % (accuracy, precision ,recall, f1_socre))

#print("\n accuracy")
#print("\n  %.4f % (accuracy)")

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 32528 samples, validate on 8133 samples
Epoch 1/500
32528/32528 [==============================] - 1s 18us/step - loss: 0.6424 - accuracy: 0.6642 - recall_m: 0.9036 - precision_m: 0.6392 - f1_m: 0.7467 - val_loss: 0.6006 - val_accuracy: 0.7001 - val_recall_m: 1.0000 - val_precision_m: 0.7003 - val_f1_m: 0.8230
Epoch 2/500
32528/32528 [==============================] - 0s 11us/step - loss: 0.5931 - accuracy: 0.7035 - recall_m: 1.0000 - precision_m: 0.7037 - f1_m: 0.8252 - val_loss: 0.5954 - val_accuracy: 0.7008 - val_recall_m: 0.9993 - val_precision_m: 0.7010 - val_f1_m: 0.8232
Epoch 3/500
32528/32528 [==============================] - 0s 11us/step - loss: 0.5889 - accuracy: 0.7049 - recall_m: 0.9973 - precision_m: 0.7053 - f1_m: 0.8254 - val_loss: 0.5918 - val_accuracy: 0.7028 - val_recall_m: 0.9876 - val_precision_m: 0.7061 - val_f1_m: 0.8226
Epoch 4/500
32528/32528 [===============

Epoch 29/500
32528/32528 [==============================] - 0s 10us/step - loss: 0.4380 - accuracy: 0.8017 - recall_m: 0.8981 - precision_m: 0.8349 - f1_m: 0.8634 - val_loss: 0.4352 - val_accuracy: 0.8071 - val_recall_m: 0.8660 - val_precision_m: 0.8596 - val_f1_m: 0.8619
Epoch 30/500
32528/32528 [==============================] - 0s 11us/step - loss: 0.4365 - accuracy: 0.8003 - recall_m: 0.8967 - precision_m: 0.8351 - f1_m: 0.8626 - val_loss: 0.4281 - val_accuracy: 0.8093 - val_recall_m: 0.8862 - val_precision_m: 0.8484 - val_f1_m: 0.8660
Epoch 31/500
32528/32528 [==============================] - 0s 10us/step - loss: 0.4297 - accuracy: 0.8074 - recall_m: 0.8973 - precision_m: 0.8414 - f1_m: 0.8670 - val_loss: 0.4294 - val_accuracy: 0.8090 - val_recall_m: 0.8642 - val_precision_m: 0.8633 - val_f1_m: 0.8629
Epoch 32/500
32528/32528 [==============================] - 0s 12us/step - loss: 0.4327 - accuracy: 0.8012 - recall_m: 0.8930 - precision_m: 0.8388 - f1_m: 0.8627 - val_loss: 0.4223

32528/32528 [==============================] - 0s 10us/step - loss: 0.4057 - accuracy: 0.8172 - recall_m: 0.8920 - precision_m: 0.8561 - f1_m: 0.8720 - val_loss: 0.3993 - val_accuracy: 0.8259 - val_recall_m: 0.9084 - val_precision_m: 0.8523 - val_f1_m: 0.8788
Epoch 60/500
32528/32528 [==============================] - 0s 10us/step - loss: 0.4094 - accuracy: 0.8145 - recall_m: 0.8943 - precision_m: 0.8521 - f1_m: 0.8707 - val_loss: 0.4099 - val_accuracy: 0.8163 - val_recall_m: 0.9257 - val_precision_m: 0.8311 - val_f1_m: 0.8751
Epoch 61/500
32528/32528 [==============================] - 0s 10us/step - loss: 0.4034 - accuracy: 0.8186 - recall_m: 0.8945 - precision_m: 0.8557 - f1_m: 0.8731 - val_loss: 0.4210 - val_accuracy: 0.8124 - val_recall_m: 0.8303 - val_precision_m: 0.8941 - val_f1_m: 0.8601
Epoch 62/500
32528/32528 [==============================] - 0s 10us/step - loss: 0.4027 - accuracy: 0.8192 - recall_m: 0.8934 - precision_m: 0.8567 - f1_m: 0.8732 - val_loss: 0.3979 - val_accura

Epoch 119/500
32528/32528 [==============================] - 0s 11us/step - loss: 0.3855 - accuracy: 0.8331 - recall_m: 0.9075 - precision_m: 0.8635 - f1_m: 0.8837 - val_loss: 0.3892 - val_accuracy: 0.8400 - val_recall_m: 0.9303 - val_precision_m: 0.8541 - val_f1_m: 0.8899
Epoch 120/500
32528/32528 [==============================] - 0s 11us/step - loss: 0.3838 - accuracy: 0.8330 - recall_m: 0.9079 - precision_m: 0.8632 - f1_m: 0.8836 - val_loss: 0.3795 - val_accuracy: 0.8404 - val_recall_m: 0.9165 - val_precision_m: 0.8635 - val_f1_m: 0.8886
Epoch 121/500
32528/32528 [==============================] - 0s 11us/step - loss: 0.3834 - accuracy: 0.8350 - recall_m: 0.9081 - precision_m: 0.8645 - f1_m: 0.8847 - val_loss: 0.3807 - val_accuracy: 0.8402 - val_recall_m: 0.9215 - val_precision_m: 0.8600 - val_f1_m: 0.8891
Epoch 122/500
32528/32528 [==============================] - 0s 11us/step - loss: 0.3866 - accuracy: 0.8330 - recall_m: 0.9083 - precision_m: 0.8628 - f1_m: 0.8836 - val_loss: 0.

Epoch 149/500
32528/32528 [==============================] - 0s 11us/step - loss: 0.3748 - accuracy: 0.8449 - recall_m: 0.9163 - precision_m: 0.8711 - f1_m: 0.8920 - val_loss: 0.3717 - val_accuracy: 0.8486 - val_recall_m: 0.9178 - val_precision_m: 0.8722 - val_f1_m: 0.8938
Epoch 150/500
32528/32528 [==============================] - 0s 11us/step - loss: 0.3706 - accuracy: 0.8473 - recall_m: 0.9176 - precision_m: 0.8722 - f1_m: 0.8932 - val_loss: 0.3652 - val_accuracy: 0.8510 - val_recall_m: 0.9140 - val_precision_m: 0.8775 - val_f1_m: 0.8948
Epoch 151/500
32528/32528 [==============================] - 0s 10us/step - loss: 0.3752 - accuracy: 0.8443 - recall_m: 0.9177 - precision_m: 0.8695 - f1_m: 0.8916 - val_loss: 0.3687 - val_accuracy: 0.8527 - val_recall_m: 0.9082 - val_precision_m: 0.8841 - val_f1_m: 0.8954
Epoch 152/500
32528/32528 [==============================] - 0s 10us/step - loss: 0.3756 - accuracy: 0.8447 - recall_m: 0.9182 - precision_m: 0.8700 - f1_m: 0.8921 - val_loss: 0.

...

Epoch 328/500
32528/32528 [==============================] - 0s 11us/step - loss: 0.3495 - accuracy: 0.8558 - recall_m: 0.9178 - precision_m: 0.8816 - f1_m: 0.8986 - val_loss: 0.3469 - val_accuracy: 0.8587 - val_recall_m: 0.9331 - val_precision_m: 0.8732 - val_f1_m: 0.9016
Epoch 329/500
32528/32528 [==============================] - 0s 11us/step - loss: 0.3535 - accuracy: 0.8511 - recall_m: 0.9146 - precision_m: 0.8795 - f1_m: 0.8957 - val_loss: 0.3461 - val_accuracy: 0.8587 - val_recall_m: 0.9196 - val_precision_m: 0.8830 - val_f1_m: 0.9003
Epoch 330/500
32528/32528 [==============================] - 0s 11us/step - loss: 0.3488 - accuracy: 0.8552 - recall_m: 0.9174 - precision_m: 0.8824 - f1_m: 0.8987 - val_loss: 0.3503 - val_accuracy: 0.8584 - val_recall_m: 0.9304 - val_precision_m: 0.8747 - val_f1_m: 0.9011
Epoch 331/500
32528/32528 [==============================] - 0s 11us/step - loss: 0.3516 - accuracy: 0.8535 - recall_m: 0.9172 - precision_m: 0.8802 - f1_m: 0.8973 - val_loss: 0.

Epoch 358/500
32528/32528 [==============================] - 0s 11us/step - loss: 0.3436 - accuracy: 0.8560 - recall_m: 0.9202 - precision_m: 0.8808 - f1_m: 0.8993 - val_loss: 0.3427 - val_accuracy: 0.8545 - val_recall_m: 0.9053 - val_precision_m: 0.8887 - val_f1_m: 0.8962
Epoch 359/500
32528/32528 [==============================] - 0s 11us/step - loss: 0.3440 - accuracy: 0.8550 - recall_m: 0.9178 - precision_m: 0.8816 - f1_m: 0.8985 - val_loss: 0.3662 - val_accuracy: 0.8532 - val_recall_m: 0.9328 - val_precision_m: 0.8671 - val_f1_m: 0.8982
Epoch 360/500
32528/32528 [==============================] - 0s 12us/step - loss: 0.3446 - accuracy: 0.8539 - recall_m: 0.9180 - precision_m: 0.8796 - f1_m: 0.8974 - val_loss: 0.3504 - val_accuracy: 0.8560 - val_recall_m: 0.9177 - val_precision_m: 0.8812 - val_f1_m: 0.8985
Epoch 361/500
32528/32528 [==============================] - 0s 11us/step - loss: 0.3416 - accuracy: 0.8552 - recall_m: 0.9177 - precision_m: 0.8813 - f1_m: 0.8983 - val_loss: 0.

Epoch 417/500
32528/32528 [==============================] - 0s 13us/step - loss: 0.3367 - accuracy: 0.8560 - recall_m: 0.9234 - precision_m: 0.8783 - f1_m: 0.8995 - val_loss: 0.3430 - val_accuracy: 0.8571 - val_recall_m: 0.9457 - val_precision_m: 0.8628 - val_f1_m: 0.9018
Epoch 418/500
32528/32528 [==============================] - 0s 12us/step - loss: 0.3394 - accuracy: 0.8563 - recall_m: 0.9245 - precision_m: 0.8778 - f1_m: 0.8996 - val_loss: 0.3394 - val_accuracy: 0.8598 - val_recall_m: 0.9392 - val_precision_m: 0.8701 - val_f1_m: 0.9028
Epoch 419/500
32528/32528 [==============================] - 0s 14us/step - loss: 0.3357 - accuracy: 0.8562 - recall_m: 0.9226 - precision_m: 0.8795 - f1_m: 0.8997 - val_loss: 0.3391 - val_accuracy: 0.8512 - val_recall_m: 0.8906 - val_precision_m: 0.8965 - val_f1_m: 0.8928
Epoch 420/500
32528/32528 [==============================] - 0s 12us/step - loss: 0.3337 - accuracy: 0.8581 - recall_m: 0.9235 - precision_m: 0.8807 - f1_m: 0.9008 - val_loss: 0.

Epoch 447/500
32528/32528 [==============================] - 0s 11us/step - loss: 0.3336 - accuracy: 0.8594 - recall_m: 0.9277 - precision_m: 0.8790 - f1_m: 0.9019 - val_loss: 0.3496 - val_accuracy: 0.8484 - val_recall_m: 0.8745 - val_precision_m: 0.9057 - val_f1_m: 0.8891
Epoch 448/500
32528/32528 [==============================] - 0s 12us/step - loss: 0.3340 - accuracy: 0.8585 - recall_m: 0.9255 - precision_m: 0.8800 - f1_m: 0.9014 - val_loss: 0.3348 - val_accuracy: 0.8582 - val_recall_m: 0.9343 - val_precision_m: 0.8718 - val_f1_m: 0.9014
Epoch 449/500
32528/32528 [==============================] - 0s 11us/step - loss: 0.3325 - accuracy: 0.8588 - recall_m: 0.9270 - precision_m: 0.8795 - f1_m: 0.9018 - val_loss: 0.3321 - val_accuracy: 0.8582 - val_recall_m: 0.9278 - val_precision_m: 0.8768 - val_f1_m: 0.9010
Epoch 450/500
32528/32528 [==============================] - 0s 11us/step - loss: 0.3363 - accuracy: 0.8567 - recall_m: 0.9254 - precision_m: 0.8782 - f1_m: 0.9003 - val_loss: 0.

Epoch 477/500
32528/32528 [==============================] - 0s 11us/step - loss: 0.3337 - accuracy: 0.8574 - recall_m: 0.9266 - precision_m: 0.8776 - f1_m: 0.9006 - val_loss: 0.3297 - val_accuracy: 0.8606 - val_recall_m: 0.9485 - val_precision_m: 0.8647 - val_f1_m: 0.9041
Epoch 478/500
32528/32528 [==============================] - 0s 12us/step - loss: 0.3310 - accuracy: 0.8594 - recall_m: 0.9294 - precision_m: 0.8779 - f1_m: 0.9022 - val_loss: 0.3297 - val_accuracy: 0.8591 - val_recall_m: 0.9333 - val_precision_m: 0.8735 - val_f1_m: 0.9019
Epoch 479/500
32528/32528 [==============================] - 0s 11us/step - loss: 0.3317 - accuracy: 0.8588 - recall_m: 0.9291 - precision_m: 0.8771 - f1_m: 0.9017 - val_loss: 0.3349 - val_accuracy: 0.8536 - val_recall_m: 0.9040 - val_precision_m: 0.8886 - val_f1_m: 0.8955
Epoch 480/500
32528/32528 [==============================] - 0s 12us/step - loss: 0.3331 - accuracy: 0.8589 - recall_m: 0.9301 - precision_m: 0.8772 - f1_m: 0.9020 - val_loss: 0.